<a href="https://colab.research.google.com/github/larasauser/2024_MLEES/blob/main/Project/project_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**# Machine learning Project - Sauser Lara**
## *Filling gaps in NDVI images using CNN and Diffusion models*

[Ajouter une image d'illustration]

##Main steps :


1.   Gather dataset
2.   Split the dataset
3.   Create the CNN alorithm
4.   Create the Dissolution model
5.   Evaluate them (RMSE ?)


##Questions
1.   Should I keep infos of date when cutting
my images ?



### 1. Loading the data and separate them into smaller images (25x25 px)
--- This should be done only once per image set because it takes time ---

--- New image set should be used after this step ---



In [1]:
#Dataset stored on Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
folder_path = '/content/drive/My Drive/NDVI_Images_Landsat8_10/'

Mounted at /content/drive


To have more images and to reduce computational cost, we decide to devide them into sub-images of 25x25 pixels.

In [2]:
pip install Pillow

In [3]:
from PIL import Image
import os

def split_image(image_path, output_folder, sub_image_size=(25, 25)):
    # Open image
    img = Image.open(image_path)
    img_width, img_height = img.size

    # Image count
    count = 0

    # Division
    for i in range(0, img_width, sub_image_size[0]):
        for j in range(0, img_height, sub_image_size[1]):
            # Cutting box
            box = (i, j, min(i + sub_image_size[0], img_width), min(j + sub_image_size[1], img_height))
            # Cutting
            sub_image = img.crop(box)
            # Verification (size)
            if sub_image.size[0] > 0 and sub_image.size[1] > 0:
                # Saving
                sub_image.save(os.path.join(output_folder, f'sub_image_{count}.tif'))
                count += 1

def process_all_images(input_folder, output_folder):
    # List all tif file in input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.tif'):
            image_path = os.path.join(input_folder, filename)
            print(f"Image in treatment : {filename}")
            split_image(image_path, output_folder)


In [4]:
#Split our images
output_folder = '/content/drive/My Drive/NDVI_Images_Landsat8_10/split'
process_all_images(folder_path, output_folder)

Image in treatment : NDVI_Val_Herens_2013-04-18.tif
Image in treatment : NDVI_Val_Herens_2013-11-12.tif
Image in treatment : NDVI_Val_Herens_2013-12-30.tif
Image in treatment : NDVI_Val_Herens_2013-11-28.tif
Image in treatment : NDVI_Val_Herens_2014-03-20.tif
Image in treatment : NDVI_Val_Herens_2014-06-08.tif
Image in treatment : NDVI_Val_Herens_2014-10-30.tif
Image in treatment : NDVI_Val_Herens_2015-02-19.tif
Image in treatment : NDVI_Val_Herens_2015-04-24.tif
Image in treatment : NDVI_Val_Herens_2015-04-08.tif
Image in treatment : NDVI_Val_Herens_2015-08-30.tif
Image in treatment : NDVI_Val_Herens_2015-05-10.tif
Image in treatment : NDVI_Val_Herens_2015-12-20.tif
Image in treatment : NDVI_Val_Herens_2016-10-03.tif
Image in treatment : NDVI_Val_Herens_2016-04-10.tif
Image in treatment : NDVI_Val_Herens_2016-12-06.tif
Image in treatment : NDVI_Val_Herens_2017-03-28.tif
Image in treatment : NDVI_Val_Herens_2017-01-23.tif
Image in treatment : NDVI_Val_Herens_2017-04-29.tif
Image in tre

NameError: name 'count' is not defined